In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, Dropout, Flatten, MaxPooling2D
import os 
import time

实现FCN-32s，其中FCN的结构如图所示：
![FCN架构](./image/FCNnetwork.png)



In [11]:
data_dir = os.path.expanduser('E:/src/dataset/VOCtrainval_11-May-2012/VOCdevkit/VOC2012')
import numpy as n
def read_images(root, train=True):
    txt_fname = root + '/ImageSets/Segmentation/' + (
        'train.txt' if train else 'val.txt')
    with open(txt_fname, 'r') as f:
        images = f.read().split()
    n = len(images)
    data, label = [None] * n, [None] * n
    for i, fname in enumerate(images):
        data[i] = cv2.imread('%s/JPEGImages/%s.jpg' % (
            root, fname))
        label[i] = cv2.imread('%s/SegmentationClass/%s.png' % (
            root, fname))
    return data, label
def rand_crop(data, label, height, width):
    for i, pic in enumerate(data):
        data[i] = cv2.resize(pic, (height, width))
    for i, lab in enumerate(label):
        label[i] = cv2.resize(lab, (height, width)) 
    return data, label

classes = ['background','aeroplane','bicycle','bird','boat',
           'bottle','bus','car','cat','chair','cow','diningtable',
           'dog','horse','motorbike','person','potted plant',
           'sheep','sofa','train','tv/monitor']
# RGB color for each class
colormap = [[0,0,0],[128,0,0],[0,128,0], [128,128,0], [0,0,128],
            [128,0,128],[0,128,128],[128,128,128],[64,0,0],[192,0,0],
            [64,128,0],[192,128,0],[64,0,128],[192,0,128],
            [64,128,128],[192,128,128],[0,64,0],[128,64,0],
            [0, 192, 0], [128, 192, 0], [0, 64, 128]]
            
cm2lbl = np.zeros(256 ** 3)
for i, cm in enumerate(colormap):
    cm2lbl[(cm[0] * 256 + cm[1]) * 256 + cm[2]] = i
    
def image2label(im):
    data = im.astype('int32').asnumpy()
    idx = (data[:,:,0]*256+data[:,:,1])*256+data[:,:,2]
    return np.array(cm2lbl[idx])

def normalize_image(data):
    rgb_mean = np.array([0.485, 0.456, 0.406])
    rgb_std = np.array([0.229, 0.224, 0.225])
    return (data.astype('float32') / 255 - rgb_mean) / rgb_std

In [7]:
import cv2
data, label = read_images(data_dir)

In [9]:
len(data)


1464

In [12]:
data, label = rand_crop(data, label, 500,500)


In [22]:
label[1].shape


(500, 500, 3)

In [19]:
from tensorflow.keras import Input
def model_fcn(input_shape=None):
    model = Sequential()
     # Block 1
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1'),input_shape=(500, 500, 1))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    # Block 2
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    # Block 3
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    # Block 4
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

    # Block 5
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

    # Convolutional layers transfered from fully-connected layers
    model.add(Conv2D(4096, (7, 7), activation='relu', padding='same', name='fc1'))
    model.add(Dropout(0.5))
    model.add(Conv2D(4096, (1, 1), activation='relu', padding='same', name='fc2'))
    model.add(Dropout(0.5))
    #classifying layer
   #  model.add(Conv2D(classes, (1, 1), kernel_initializer='he_normal', activation='linear', padding='valid', strides=(1, 1))(x)
    model.add(Conv2D(classes, (1,1), activation = 'sigmoid'))
    
    model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc'])
    return model
model = model_fcn(data[0].shape)

ValueError: Layer block1_conv1 expects 1 inputs, but it received 3 input tensors. Inputs received: [500, 500, 3]